In [173]:
# for data manipulation
import pandas as pd
import numpy as np
# for plotting
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go # note that github wont be able to display these plots because they are interactive
# for some processing
import math
from datetime import datetime, timedelta

In [174]:
folder = "kaggle/input/"
df_train = pd.read_csv(folder + "train.csv")
df_train

,CustomerID,State,Customer Lifetime Value,Response,Coverage,Coverage Index,Education,Education Index,Effective To Date,Employment Status,...,Policy Type,Policy Type Index,Policy,Policy Index,Renew Offer Type,Sales Channel,Sales Channel Index,Vehicle Size,Vehicle Size Index,Claim over 1k
0,QC35222,California,3622.69,No,Basic,0,Bachelor,2,1/1/2024,Employed,...,Corporate Auto,1,Corporate L2,4,3,Web,0,Medsize,1,0
1,AE98193,Washington,10610.21,No,Basic,0,High School or Below,0,1/1/2024,Unemployed,...,Personal Auto,0,Personal L1,0,1,Branch,1,Medsize,1,1
2,TM23514,Oregon,13868.02,No,Extended,1,College,1,1/1/2024,Employed,...,Personal Auto,0,Personal L3,2,3,Web,0,Medsize,1,0
3,QZ42725,Washington,3119.69,No,Basic,0,Bachelor,2,1/1/2024,Unemployed,...,Personal Auto,0,Personal L3,2,2,Agent,2,Medsize,1,0
4,SG81493,Arizona,5999.04,No,Premium,2,Bachelor,2,1/1/2024,Employed,...,Corporate Auto,1,Corporate L1,3,2,Web,0,Medsize,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7285,HC90344,California,27500.54,No,Basic,0,Bachelor,2,9/2/2024,Unemployed,...,Corporate Auto,1,Corporate L2,4,1,Branch,1,Medsize,1,0
7286,ZU83252,California,11750.03,No,Basic,0,Master,3,9/2/2024,Employed,...,Personal Auto,0,Personal L3,2,3,Web,0,Medsize,1,0
7287,PR80703,California,7757.04,No,Basic,0,Bachelor,2,9/2/2024,Employed,...,Personal Auto,0,Personal L3,2,1,Branch,1,Medsize,1,0
7288,NS23754,California,3465.16,No,Basic,0,High School or Below,0,9/2/2024,Employed,...,Corporate Auto,1,Corporate L3,5,3,Call Center,3,Large,2,0


In [175]:
df_train = df_train.drop(columns = ["Coverage", "Education", "Employment Status", "Marital Status", "Policy Type", "Policy", "Sales Channel", "Vehicle Size"])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7290 entries, 0 to 7289
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CustomerID                     7290 non-null   object 
 1   State                          7290 non-null   object 
 2   Customer Lifetime Value        7290 non-null   float64
 3   Response                       7290 non-null   object 
 4   Coverage Index                 7290 non-null   int64  
 5   Education Index                7290 non-null   int64  
 6   Effective To Date              7290 non-null   object 
 7   Employment Status Index        7290 non-null   int64  
 8   Gender                         7290 non-null   object 
 9   Income                         7290 non-null   int64  
 10  Marital Status Index           7290 non-null   int64  
 11  Months Since Last Claim        7290 non-null   int64  
 12  Months Since Policy Inception  7290 non-null   i

In [176]:
def process_state(df):
  unique_states = df["State"].unique()
  dict_states = {}
  for i in range(len(unique_states)):
    dict_states[unique_states[i]] = i
  df["State"] = df["State"].apply(lambda x: dict_states[x])

In [177]:
process_state(df_train)

In [178]:
def numeric_info(df, col):
  print(df[col].mean())
  print(df[col].median())
  print(df[col].std())

In [179]:
numeric_info(df_train, "Customer Lifetime Value")

10798.11662277092
7845.015
9180.878152795805


In [180]:
def plot(bins):
  df_cp = df_train.copy()
  df_cp["value_bins"] = pd.cut(df_cp["Customer Lifetime Value"], bins = bins, right = False)
  counts = df_cp["value_bins"].value_counts().reindex(df_cp["value_bins"].cat.categories)
  fig = px.bar(
    df_cp,
    x = counts.index.astype(str),
    y = counts.values,
  )
  fig.show()
  print(counts)
  print(type(counts))
plot([0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000])
  

[0, 10000)         4456
[10000, 20000)     2054
[20000, 30000)      432
[30000, 40000)      195
[40000, 50000)       95
[50000, 60000)       32
[60000, 70000)       13
[70000, 80000)        6
[80000, 90000)        5
[90000, 100000)       1
Name: count, dtype: int64
<class 'pandas.core.series.Series'>


In [181]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score

In [182]:
scaler = StandardScaler()
df_train["Customer Lifetime Value"] = scaler.fit_transform(df_train[["Customer Lifetime Value"]])

In [183]:
plot([-10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
numeric_info(df_train, "Customer Lifetime Value")

[-10, 0)    4836
[0, 1)      1673
[1, 2)       406
[2, 3)       207
[3, 4)        89
[4, 5)        48
[5, 6)        13
[6, 7)         7
[7, 8)         7
[8, 9)         2
[9, 10)        2
Name: count, dtype: int64
<class 'pandas.core.series.Series'>
-9.551877654936875e-17
-0.3216799242025451
1.000068594162716


In [185]:
df_train["Effective To Date"] = pd.to_datetime(df_train["Effective To Date"])
today = pd.Timestamp.today()
df_train["Number of Days"] = (today - df_train["Effective To Date"]).dt.days
print(df_train["Number of Days"].value_counts())

numeric_info(df_train, "Number of Days")

Number of Days
 5009    154
 13      151
 5005    149
 5017    148
 5019    147
 227     144
 166     143
 5010    141
 4991    141
 5008    140
 5016    139
 4978    138
 226     134
 256     133
-18      133
 4977    132
 12      131
 195     130
 5007    128
 104     127
 4983    127
 4979    125
 5011    124
 105     123
 4986    123
 5022    123
 4982    122
-49      122
 43      122
 165     121
 74      121
 286     121
 4987    120
 135     120
 4980    119
 5015    118
 5006    118
 255     118
 5021    117
 5013    117
 287     116
 5018    116
 5012    116
-19      116
 4992    114
 5020    114
 4981    112
 4984    112
 42      112
 5014    111
 4988    111
 134     110
 4989    110
 73      108
 5023    107
 4985    107
 4990    101
-48       99
 196      94
Name: count, dtype: int64
3026.4694101508917
4982.0
2396.505089086872


In [186]:
df_train["Number of Days"] = scaler.fit_transform(df_train[["Number of Days"]])

In [187]:
numeric_info(df_train, "Number of Days")

-3.118980458754898e-17
0.8160486438097844
1.0000685941627134


In [188]:
df_train["Gender"] = df_train["Gender"].apply(lambda x: 1 if x == "M" else 0)

In [189]:
numeric_info(df_train, "Income")

50573.28257887517
45398.5
41090.96814180818


In [190]:
df_train["Income"] = scaler.fit_transform(df_train[["Income"]])

In [191]:
numeric_info(df_train, "Income")

2.826576040746626e-17
-0.12594343167806607
1.0000685941627347


In [192]:
df_train["Months Since Last Claim"].value_counts()

Months Since Last Claim
4     304
8     291
1     273
9     264
7     259
5     258
3     256
14    243
0     241
18    237
22    233
15    230
20    226
11    222
16    217
19    212
23    205
12    197
26    195
27    195
31    193
34    187
28    187
24    184
30    183
39    173
38    167
32    161
41    152
42    152
35    151
36    143
45    141
46    136
47    111
43    111
Name: count, dtype: int64

In [193]:
numeric_info(df_train, "Months Since Last Claim")

20.514677640603566
19.0
13.62641170038164


In [194]:
minmax_scaler = MinMaxScaler()
df_train["Months Since Last Claim"] = minmax_scaler.fit_transform(df_train[["Months Since Last Claim"]])

In [195]:
numeric_info(df_train, "Months Since Last Claim")

0.43648250299156527
0.40425531914893614
0.28992365319960745


In [196]:
df_train["Months Since Policy Inception"].value_counts()

Months Since Policy Inception
80     118
68     104
82     103
14      96
59      93
      ... 
127     54
8       47
111     46
132     42
131     41
Name: count, Length: 100, dtype: int64

In [197]:
numeric_info(df_train, "Months Since Policy Inception")

65.16433470507545
65.0
37.65612057913027


In [198]:
df_train["Months Since Policy Inception"] = minmax_scaler.fit_transform(df_train[["Months Since Last Claim"]])

In [199]:
numeric_info(df_train, "Months Since Policy Inception")

0.43648250299156527
0.40425531914893614
0.28992365319960745


In [200]:
df_train["Number of Open Complaints"].value_counts()

Number of Open Complaints
0    5798
1     809
2     299
3     225
4     112
5      47
Name: count, dtype: int64

In [202]:
df_train["Number of Open Complaints"] = minmax_scaler.fit_transform(df_train[["Number of Open Complaints"]])

In [203]:
numeric_info(df_train, "Number of Open Complaints")

0.07585733882030178
0.0
0.180672244949201


In [204]:
df_train["Number of Policies"].value_counts()

Number of Policies
1    2589
2    1836
3     921
7     353
9     339
4     325
5     320
8     314
6     293
Name: count, dtype: int64

In [205]:
df_train["Number of Policies"] = minmax_scaler.fit_transform(df_train[["Number of Policies"]])

In [206]:
numeric_info(df_train, "Number of Policies")

0.24735939643347052
0.125
0.3005286804508821


KeyError: 'Claim Over 1k'